## Initialize Notebook

In [0]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

In [0]:
# Initialize needed variables
root = '/content/gdrive/My Drive/XSible/'
voc_dataset = root + 'pseudo-voc-dataset'

## Install dependencies

In [0]:
!pip install torch torchvision
!pip install xmltodict
!pip install opencv-contrib-python

## Clone SSD repository

In [0]:
# Remove old repository files
!cd '/content/'
!rm -rf 'ssd.pytorch/'

In [0]:
# Clone repository
!git clone https://github.com/xsible/ssd.pytorch

## Fetch VGG16 weights

In [0]:
!mkdir -p model-weights/
!wget https://s3.amazonaws.com/amdegroot-models/vgg16_reducedfc.pth -P '/content/model-weights/'

## SSD model with VOC dataset

### Fetch Pascal VOC dataset

In [0]:
# download sample data from VOC 2007 & 2012 dataset
!mkdir -p '/content/voc-data/'
!sh ssd.pytorch/data/scripts/VOC2007.sh '/content/voc-data/'
!sh ssd.pytorch/data/scripts/VOC2012.sh '/content/voc-data/'

### Train model

In [0]:
# Delete current model weights
!rm -rf model-weights/VOC.pth

# Train new model
!python ssd.pytorch/train.py --dataset_root /content/og-data/VOCdevkit/ --save_folder /content/model-weights/ --start_iter 119950

### Evaluate model

In [0]:
# Evaluate standard model
!python ssd.pytorch/eval.py --voc_root /content/og-data/VOCdevkit/ --trained_model model-weights/VOC.pth --max_test 50

## SSD model with custom dataset

### Train model

In [0]:
# Delete current model weights
!rm -rf model-weights/custom.pth

# Train new model
!python ssd.pytorch/train.py --dataset custom --batch_size 8 --dataset_root '/content/gdrive/My Drive/XSible/pseudo-voc-dataset/' --save_folder /content/model-weights/ --start_iter 35

### Evaluate model

In [0]:
# Evaluate custom model
!python ssd.pytorch/eval.py --use_07_metric false --year 2019 --use_custom true --voc_root '/content/gdrive/My Drive/XSible/pseudo-voc-dataset/' --trained_model /content/model-weights/custom.pth --max_test 50

### Test model

In [0]:
# Test custom model
!rm eval/test1.txt
!python ssd.pytorch/test.py -f 0 --use_custom true --voc_root '/content/gdrive/My Drive/XSible/pseudo-voc-dataset/' --trained_model model-weights/custom.pth --visual_threshold 0.95

### Run inference

In [0]:
!python ssd.pytorch/inference.py --input '/content/gdrive/My Drive/XSible/pseudo-voc-dataset/VOC2019/JPEGImages/img_275.jpg' --trained_model model-weights/custom.pth